In [74]:
import h5py
from typing import Protocol, Any, Union, Dict, List
import pathlib
from h5rdmtoolbox.x2hdf.piv import DEFAULT_CONFIGURATION

In [75]:
class PIVFile:
    """Adapter class for any conversion of PIV data"""

In [52]:
class PIVSnapshot:
    """Interface class"""
    
    def __init__(self, piv_file: PIVFile, recording_time: float):
        if not isinstance(piv_file, PIVFile):
            raise TypeError(f'Expecting type {PIVFile.__class__}, not type {type(piv_file)}')
        self.piv_file = piv_file
        self.recording_time = recording_time
        
    def to_hdf(self, hdf_filename):
        """converts the snapshot into an HDF file"""
        self.piv_file.to_hdf(hdf_filename)
        
class PIVPlane:
    """Interface class"""
    
    def __init__(self, list_of_piv_file: List[PIVFile]):
        if not isinstance(list_of_piv_file, (tuple, list)):
            raise TypeError(f'Expecting a list of {PIVFile.__class__} objects but got {type(list_of_piv_file)}')
        if not all([isinstance(piv_file, PIVFile) for piv_file in list_of_piv_file]):
            for piv_file in list_of_piv_file:
                if not isinstance(piv_file, PIVFile):
                    raise TypeError(f'Expecting type {PIVFile.__class__} for each entry, but one entry is of type {type(piv_file)}')
        self.list_of_piv_file = list_of_piv_file
        
    def to_hdf(self, hdf_filename):
        """converts the snapshot into an HDF file"""
        for piv_file in self.list_of_piv_file:
            piv_file.to_hdf(hdf_filename)

In [67]:
class PivViewNcFile(PIVFile):
    
    def __init__(self, filename: pathlib.Path):
        _filename = pathlib.Path(filename)
        if not filename.is_file:
            raise TypeError(f'Snapshot file path is not a file: {_filename}.')
        if filename.suffix != '.nc':
            raise NameError(f'Expecting a netCDF4 file, thus suffix .nc, not {_filename.suffix}')
        self.filename = _filename
        
    def _get_nc_data(self, build_coord_datasets=True):
        masking = self.configuration['masking']
        interpolation = self.configuration['interpolation']
        apply_mask = self.configuration['apply_mask']
        z_source = core.PIV_Z_Source(self.configuration['z_source']).value
        return process_pivview_nc_data(self.name, z_source=z_source,
                                       timestep=self.recording_time,
                                       masking=masking,
                                       interpolate=interpolation,
                                       apply_mask=apply_mask,
                                       build_coord_datasets=build_coord_datasets,
                                       standardized_name_table=self.configuration['standardized_name_table'])
        
    def to_hdf(self, hdf_filename):
        """converts the snapshot into an HDF file"""
        print(f'converting {self.filename.name}...')

In [68]:
import h5rdmtoolbox as h5tbx

In [69]:
nc_file = h5tbx.tutorial.PIVview.get_snapshot_nc_files()[0]

In [70]:
PivViewNcFile(nc_file)

In [71]:
snapshot = PIVSnapshot(PivViewNcFile(nc_file))
snapshot.to_hdf('test.hdf')

In [72]:
snapshot.to_hdf('test.hdf')

converting E00A1.nc...


In [73]:
plane = PIVPlane([PivViewNcFile(nc_file), PivViewNcFile(nc_file)])
plane.to_hdf('test.hdf')

converting E00A1.nc...
converting E00A1.nc...


{'interpolation': False,
 'apply_mask': True,
 'masking': 'slack',
 'time_unit': 's',
 'z_source': 'coord_min',
 'datetime_str': '%Y-%m-%dT%H:%M:%SZ%z',
 'attrs_unit_name': 'units',
 'compression': 'gzip',
 'compression_opts': 5,
 'take_min_nt': True,
 'standardized_name_table': piv (version number: None),
 'timeAverages': {'compute': True, 'use_nc': False},
 'post': {'compute': True,
  'grpname': 'post',
  'grpdesc': 'Post processing data',
  'running_mean': {'compute': True,
   'grpname': 'running_mean',
   'grpdesc': 'Running mean',
   'dataset_names': ['u', 'v']},
  'running_std': {'compute': True,
   'grpname': 'running_std',
   'grpdesc': 'Running standard deviation',
   'dataset_names': ['u', 'v']},
  'velocity_abs_ds_name': 'c',
  'compute_dwdz': False}}

In [98]:
from h5rdmtoolbox.x2hdf.piv.utils import PIVviewFlag

In [84]:
print(PIVviewFlag(0))

inactive


In [125]:
from enum import IntEnum

In [126]:
_hex_dict = {"inactive": "0x0", "active": "0x1", "masked": "0x2",
                 "noresult": "0x4", "disabled": "0x8", "filtered": "0x10",
                 "interpolated": "0x20", "replaced": "0x40", "manualedit": "0x80"}

In [127]:
int('0x0',16)

0

In [128]:
class MyPIVviewFlag(IntEnum):
    inactive = int('0x0',16), 16
    active = int('0x1',16), 16
    masked = int('0x2',16), 16
    noresult = int('0x4',16), 16
    disabled = int('0x8',16), 16
    filtered = int('0x10',16), 16
    interpolated = int('0x20',16), 16
    replaced = int('0x40',16), 16
    manualedit = int('0x80',16), 16

TypeError: int() can't convert non-string with explicit base

In [129]:
MyPIVviewFlag.inactive

<MyPIVviewFlag.inactive: '0x0'>

In [101]:
print(PIVviewFlag(10))

masked+disabled


In [111]:
hex(8)

'0x8'

In [114]:
for i in range(100):
    try:
        print(MyPIVviewFlag(hex(i)))
    except:
        pass

MyPIVviewFlag.inactive
MyPIVviewFlag.active
MyPIVviewFlag.masked
MyPIVviewFlag.noresult
MyPIVviewFlag.disabled
MyPIVviewFlag.filtered
MyPIVviewFlag.interpolated
MyPIVviewFlag.replaced


In [116]:
MyPIVviewFlag.active | MyPIVviewFlag.masked

TypeError: unsupported operand type(s) for |: 'str' and 'str'